<a href="https://colab.research.google.com/github/barzansaeedpour/advanced-python-notebooks/blob/main/SOLID_Principles_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLID Principles with Python
<div dir="rtl">
<h1>
اصول SOLID با پایتون
</h1>
</div>

<div dir="rtl">
اصول SOLID در برنامه نویسی توسط آقای «رابرت سی. مارتین» (Robert C. Martin) در مقاله‌ای به نام «اصول طراحی و الگوهای طراحی» (Design Principles and Design Patterns) در سال ۱۳۸۲ شمسی (۲۰۰۰ میلادی) معرفی شدند. این مفاهیم بعدا توسط «مایکل فدرز» (Michael Feathers) گسترش یافتند. آقای فدرز کسی بود که کلمه مخفف «سالید» (SOLID) را ابدع کرد و برای اولین بار استفاده کرد و در طول ۲۰ سال گذشته، این اصول پنج‌گانه از طریق اصلاح راه و روش نوشتن نرم‌افزارها، دنیای برنامه‌نویسی شی‌گرایانه را دچار تغییر و تحول بزرگی کردند. در این نوت بوک به اصول سالید در برنامه نویسی خواهیم پرداخت و برای هریک با نوشتن نمونه کد مثال‌هایی ارائه می‌دهیم.
</div>

<div dir="rtl">
    <ul>
        <li>
            اصل تک‌مسئولیتی (Single Responsibility Principle | SRP)
        </li>
        <li>
            اصل باز - بسته (Open/Closed Principle | OCP)
        </li>
        <li>
            اصل جایگزینی لیسکوف (Liskov Substitution Principle | LSP)
        </li>
        <li>
            اصل جداسازی اینترفیس‌ها (Interface Segregation Principle | ISP)
        </li>
        <li>
            اصل وارونگی وابستگی (Dependency Inversion Principle | DIP)
        </li>
    </ul>
</div>

## Resources
## منابع

- https://medium.com/@m.nusret.ozates/solid-principles-with-python-245e45f9b1f8
- https://blog.faradars.org/%D8%A7%D8%B5%D9%88%D9%84-solid-%D8%AF%D8%B1-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3%DB%8C/